In [ ]:
%matplotlib notebook

In [ ]:
from nilearn import plotting, datasets, masking
import nibabel as nib
import os

## MNI anatomy

In [ ]:
from nilearn.datasets import load_mni152_template
template = load_mni152_template()
print(template)
plotting.plot_anat(template)

## Your volume

In [ ]:
import numpy as np
vol = nib.load('sub-10_T1w.nii.gz')
img = vol.get_data().swapaxes(0,2)
tranf_vol = nib.Nifti1Image(img, np.eye(4))
plotting.plot_anat(vol, title = "Original")
plotting.plot_anat(tranf_vol, title = "Transformed")

In [ ]:
from nipype.interfaces import fsl
btr = fsl.BET()
btr.inputs.in_file = 'sub-10_T1w.nii.gz'
btr.inputs.frac = 0.7
btr.inputs.output_type = 'NIFTI_GZ'
btr.inputs.out_file = 'output/fsl/brain_anat.nii.gz'
res = btr.run() 


In [ ]:
plotting.plot_roi('./output/fsl/brain_anat.nii.gz', bg_img = 'sub-10_T1w.nii.gz', title = "Extracted brain mask")

## ANTs registration to mni

ANTs allows us to register (transform, warp, deform) a volume toward another. Usually, a given volume acquired with MRI to a template, such as mn152.

In [ ]:
from nipype.interfaces.ants import Registration
reg = Registration()
reg.inputs.fixed_image = os.path.join(datasets.__path__[0], "data","avg152T1_brain.nii.gz")
reg.inputs.moving_image = 'sub-10_T1w.nii.gz'
reg.inputs.output_transform_prefix = "output/ants/output_"
#reg.inputs.initial_moving_transform = 'trans.mat'
reg.inputs.transforms = ['Affine', 'SyN']
reg.inputs.transform_parameters = [(2.0,), (0.25, 3.0, 0.0)]
reg.inputs.number_of_iterations = [[1500, 200], [100, 50, 30]]
reg.inputs.dimension = 3
reg.inputs.write_composite_transform = True
reg.inputs.collapse_output_transforms = False
reg.inputs.initialize_transforms_per_stage = False
reg.inputs.metric = ['Mattes']*2
reg.inputs.metric_weight = [1]*2 # Default (value ignored currently by ANTs)
reg.inputs.radius_or_number_of_bins = [32]*2
reg.inputs.sampling_strategy = ['Random', None]
reg.inputs.sampling_percentage = [0.05, None]
reg.inputs.convergence_threshold = [1.e-8, 1.e-9]
reg.inputs.convergence_window_size = [20]*2
reg.inputs.smoothing_sigmas = [[1,0], [2,1,0]]
reg.inputs.sigma_units = ['vox'] * 2
reg.inputs.shrink_factors = [[2,1], [3,2,1]]
reg.inputs.use_estimate_learning_rate_once = [True, True]
reg.inputs.use_histogram_matching = [True, True] # This is the default
reg.inputs.output_warped_image = 'output/ants/output_warped_image.nii.gz'
reg.run()  

In [ ]:
plotting.plot_roi('./output/ants/output_warped_image.nii.gz', bg_img = 'sub-10_T1w.nii.gz', title = "Extracted brain mask")


## FSL

In [ ]:
from nipype.interfaces import fsl
first = fsl.FIRST()
first.inputs.in_file = 'sub-10_T1w.nii.gz'
first.inputs.output_type = 'NIFTI_GZ'
#first.inputs.out_file = './output/fsl/segmented.nii.gz'
first.out_basename = './output/fsl/sub-10_T1w_'
print(first.cmdline)
print(first)
res = first.run() 

In [ ]:
destrieux_atlas = datasets.fetch_atlas_destrieux_2009()
destrieux_parcellation = destrieux_atlas['maps']
plotting.plot_roi(destrieux_parcellation, bg_img = './output/ants/output_warped_image.nii.gz', cmap='gist_ncar',
                  title='Destrieux 2010 parcellation')